In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sfp_nsdsyn.two_dimensional_model as two_d
import sfp_nsdsyn.cross_validation_2d_model as cv2d
import warnings
from sfp_nsdsyn import utils
import itertools
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

In [2]:
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
dset='nsdsyn'
subj='subj01'
roi='V1'
vs='pRFsize'
df = utils.load_dataframe(output_dir, dset, subj, roi, vs)
df = df.groupby(['sub', 'voxel', 'class_idx', 'names','vroinames']).mean().reset_index()

In [86]:
sfp_model = two_d.SpatialFrequencyModel()
cv_results, cv_losses = cv2d.run_cross_validation(df, sfp_model, 
                                               n_folds=7, 
                                               n_test_classes=4, 
                                               learning_rate=1e-4, 
                                               max_epoch=2, 
                                               print_every=100, 
                                               random_state=42)



Fold 1/7
**epoch no.0 loss: 0.044
*epoch no.2: Finished! final model params...
 {'sigma': 0.78, 'slope': 0.34, 'intercept': 0.94, 'p_1': 0.01, 'p_2': 0.09, 'p_3': 0.02, 'p_4': 0.05, 'A_1': 0.07, 'A_2': 0.4}
Elapsed time: 0.02 sec

Fold 2/7
**epoch no.0 loss: 0.041
*epoch no.2: Finished! final model params...
 {'sigma': 0.78, 'slope': 0.34, 'intercept': 0.94, 'p_1': 0.01, 'p_2': 0.09, 'p_3': 0.02, 'p_4': 0.05, 'A_1': 0.07, 'A_2': 0.4}
Elapsed time: 0.02 sec

Fold 3/7
**epoch no.0 loss: 0.042
*epoch no.2: Finished! final model params...
 {'sigma': 0.78, 'slope': 0.34, 'intercept': 0.94, 'p_1': 0.01, 'p_2': 0.09, 'p_3': 0.02, 'p_4': 0.05, 'A_1': 0.07, 'A_2': 0.4}
Elapsed time: 0.02 sec

Fold 4/7
**epoch no.0 loss: 0.043
*epoch no.2: Finished! final model params...
 {'sigma': 0.78, 'slope': 0.34, 'intercept': 0.94, 'p_1': 0.01, 'p_2': 0.09, 'p_3': 0.02, 'p_4': 0.05, 'A_1': 0.07, 'A_2': 0.4}
Elapsed time: 0.02 sec

Fold 5/7
**epoch no.0 loss: 0.039
*epoch no.2: Finished! final model params

In [90]:
cv_losses.detach().numpy()

array([0.02863195, 0.04803467, 0.04623995, ..., 0.05495894, 0.0490094 ,
       0.05865856])

In [100]:
max_epoch = 30000
nsd_subj_list = ['subj01', 'subj02', 'subj03', 'subj04', 'subj05', 'subj06', 'subj07', 'subj08']

loss_df = pd.DataFrame({})
for subj in nsd_subj_list[:2]:
    train_loss = pd.read_hdf(f'/Volumes/server/Projects/sfp_nsd/derivatives/sfp_model/results_2D/nsdsyn/corrected/loss-history_lr-0.0005_eph-30000_sub-{subj}_roi-V1_vs-pRFsize.h5')
    test_loss = np.load(f'/Volumes/server/Projects/sfp_nsd/derivatives/sfp_model/cross_validation/nsdsyn/model-1/cvlosses_bts-False_lr-0.0005_eph-30000_sub-{subj}_roi-V1_vs-pRFsize.npy')
    tmp = pd.DataFrame({'subj': [subj], 
                        'train_mean_loss': [train_loss.query('epoch == @max_epoch-1')['loss'].item()],
                        'test_mean_loss': [test_loss.mean()],
                        })
    loss_df = pd.concat([loss_df, tmp], axis=0)





In [102]:
pd.read_csv('/Volumes/server/Projects/sfp_nsd/derivatives/sfp_model/cross_validation/nsdsyn/model-1/cvresults_bts-False_lr-0.0005_eph-30000_sub-subj01_roi-V1_vs-pRFsize.csv')

,fold,test_classes,sigma,slope,intercept,p_1,p_2,p_3,p_4,A_1,A_2
0,0,[18 16 3 21],2.528584,0.135677,0.273609,-0.094468,0.024804,-0.192949,-0.007711,0.085125,-0.020095
1,1,[ 5 6 2 27],2.381408,0.172235,0.217421,-0.051729,0.016940,-0.289116,0.044692,0.077384,-0.011702
2,2,[12 7 10 17],2.653118,0.125977,0.341817,-0.100426,-0.010608,-0.184935,0.045035,0.087633,-0.013171
3,3,[ 0 26 25 14],2.450733,0.154401,0.244995,-0.067695,0.012135,-0.305609,0.035673,0.072337,-0.013143
4,4,[23 24 19 8],2.558670,0.140593,0.284998,-0.055584,-0.020705,-0.315272,0.039840,0.085760,-0.013825
5,5,[ 9 4 1 15],2.554207,0.132607,0.261325,-0.056628,0.000472,-0.179138,-0.042741,0.070708,-0.022408
6,6,[11 22 13 20],2.685035,0.105868,0.318183,-0.094474,-0.009001,-0.311549,0.045119,0.084734,-0.014832


In [101]:
loss_df

,subj,train_mean_loss,test_mean_loss
0,subj01,0.003005,0.003188
0,subj02,0.004129,0.004354


In [ ]:
sns.set_context("notebook")
sample_df = loss_df.melt(id_vars=['sub'], 
                        var_name='loss_type', 
                        value_name='loss')
fig = plt.figure(figsize=(8, 13), dpi=300)
# Create GridSpec with custom width ratios for top row
gs = gridspec.GridSpec(3, 2, width_ratios=[1, 2], height_ratios=[1, 1, 1])

# Create axes using the GridSpec
axes = np.empty((2, 2), dtype=object)
axes[0, 0] = fig.add_subplot(gs[0, 0])  # larger
# Plot 1: Train vs Test loss across folds & subjects
sns.barplot(ax=axes[0, 0], 
            data=sample_df, 
            x='loss_type', y='loss', hue='loss_type', errorbar=('ci', 68))

0.0030051774015131738